In [14]:

import cantera as ct
import numpy as np

#Parameters
T=383 #Initial temperature in K
p=1 #pressure in bar
volume = 0.1e-6 #volume of the catalyst zone
beta=20.0 #temperature ramp in K/min
m_cat=0.0413 #catalyst mass in g
cat_area = 10.4 #area in m2/g 
facet_fraction=[0.6923, 0.0439, 0.211, 0.0528] # [111,211, 100, 110]
sccm = 30.652*1e-6/60 #Flow rate m3/s

# input file containing the surface reaction mechanism
cti_file = 'original_mechanism_JPCC_2021/Multifacet_mechanism.yaml'

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')
gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

#import the (111) surface
surf111 = ct.Interface(cti_file,'surface1', [gas])
surf111.TP = T, p*ct.one_atm
surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

#import the (211) surface
surf211 = ct.Interface(cti_file,'surface2', [gas])
surf211.TP = T, p*ct.one_atm
surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

#import the (100) surface
surf100 = ct.Interface(cti_file,'surface3', [gas])
surf100.TP = T, p*ct.one_atm
surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

#import the (110) surface
surf110 = ct.Interface(cti_file,'surface4', [gas])
surf110.TP = T, p*ct.one_atm
surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

#typical Cantera setup
upstream = ct.Reservoir(gas, name='upstream')
downstream = ct.Reservoir(gas, name='downstream')

r=ct.IdealGasReactor(gas, energy='off')
r.volume=volume
rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

#This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
mass_flow_rate =  vdot* gas.density
mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

sim=ct.ReactorNet([r])

##set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-22
sim.max_err_test_fails = 10000
rxn_time=np.linspace(0,8400,8400)

#get indices of the gas and surface species, (211) and (111) have the same order of species
i_co2=gas.species_index('CO2')
i_co=gas.species_index('CO')

i_surf_Ni=surf111.species_index('site(111)')
i_surf_CO=surf111.species_index('OC(111)')
i_surf_O=surf111.species_index('O(111)')
i_surf_C=surf111.species_index('C(111)')
i_surf_CO2=surf111.species_index('CO2(111)')

gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])

Ti=np.zeros(len(rxn_time))
sim.max_time_step=1e-4
#Run the simulation
for i in range(len(rxn_time)):
        time=rxn_time[i]
        print(time)
        if time < 10:
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
            upstream.syncState()
        elif (time > 10) and (time < 3610):   
            sim.max_time_step=1
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
            upstream.syncState()
        elif (time > 3610) and (time < 3970): 
            sim.max_time_step=10
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
            upstream.syncState()
        elif (time > 3970) and (time < 5410):
            sim.max_time_step=10
            upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
            upstream.syncState()
            r.thermo.TP=323,ct.one_atm
            r.syncState()
        else:
            r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
            r.syncState()
             
        sim.reinitialize  
        sim.advance(time)
        Ti[i]=r.thermo.T
        gas_mole_fracs[:,i]=gas.X
        
#Get the maximum and the position of the maximum of the TPD profile
ind=5410

ref_height_CO2=np.max(gas_mole_fracs[i_co2, ind:])*1e6
ref_Temp_CO2=Ti[np.argmax(gas_mole_fracs[i_co2,ind:])+5410]

ref_height_CO=np.max(gas_mole_fracs[i_co, ind:])*1e6
ref_Temp_CO=Ti[np.argmax(gas_mole_fracs[i_co,ind:])+5410]

0.0
1.0001190617930706
2.0002381235861413
3.0003571853792117
4.000476247172283
5.000595308965353
6.000714370758423
7.000833432551494
8.000952494344565
9.001071556137635
10.001190617930707
11.001309679723777
12.001428741516847
13.001547803309919
14.001666865102989
15.00178592689606
16.00190498868913
17.002024050482202
18.00214311227527
19.002262174068342
20.002381235861414
21.002500297654482
22.002619359447554
23.002738421240625
24.002857483033694
25.002976544826765
26.003095606619837
27.00321466841291
28.003333730205977
29.00345279199905
30.00357185379212
31.00369091558519
32.00380997737826
33.00392903917133
34.004048100964404
35.004167162757476
36.00428622455054
37.00440528634361
38.004524348136684
39.004643409929756
40.00476247172283
41.0048815335159
42.005000595308964
43.005119657102036
44.00523871889511
45.00535778068818
46.00547684248125
47.00559590427432
48.00571496606739
49.00583402786046
50.00595308965353
51.0060721514466
52.006191213239674
53.006310275032746
54.00642933682582


849.101083462317
850.10120252411
851.1013215859031
852.1014406476962
853.1015597094893
854.1016787712823
855.1017978330754
856.1019168948685
857.1020359566616
858.1021550184546
859.1022740802476
860.1023931420408
861.1025122038338
862.1026312656269
863.1027503274199
864.1028693892131
865.1029884510061
866.1031075127992
867.1032265745922
868.1033456363854
869.1034646981784
870.1035837599715
871.1037028217645
872.1038218835575
873.1039409453507
874.1040600071437
875.1041790689368
876.1042981307298
877.104417192523
878.104536254316
879.1046553161091
880.1047743779021
881.1048934396953
882.1050125014883
883.1051315632814
884.1052506250744
885.1053696868676
886.1054887486606
887.1056078104536
888.1057268722467
889.1058459340398
890.1059649958329
891.1060840576259
892.106203119419
893.1063221812121
894.1064412430052
895.1065603047982
896.1066793665913
897.1067984283843
898.1069174901775
899.1070365519705
900.1071556137636
901.1072746755566
902.1073937373498
903.1075127991428
904.107631860935

1951.2322895582809
1952.2324086200738
1953.232527681867
1954.23264674366
1955.232765805453
1956.2328848672462
1957.2330039290393
1958.2331229908323
1959.2332420526254
1960.2333611144184
1961.2334801762115
1962.2335992380047
1963.2337182997976
1964.2338373615908
1965.233956423384
1966.2340754851768
1967.23419454697
1968.234313608763
1969.234432670556
1970.2345517323492
1971.2346707941422
1972.2347898559353
1973.2349089177283
1974.2350279795214
1975.2351470413146
1976.2352661031075
1977.2353851649007
1978.2355042266938
1979.2356232884867
1980.23574235028
1981.2358614120728
1982.235980473866
1983.2360995356592
1984.236218597452
1985.2363376592452
1986.2364567210384
1987.2365757828313
1988.2366948446245
1989.2368139064174
1990.2369329682106
1991.2370520300037
1992.2371710917967
1993.2372901535898
1994.237409215383
1995.237528277176
1996.237647338969
1997.237766400762
1998.2378854625551
1999.2380045243483
2000.2381235861412
2001.2382426479344
2002.2383617097273
2003.2384807715205
2004.23859

2801.333492082391
2802.333611144184
2803.333730205977
2804.33384926777
2805.333968329563
2806.334087391356
2807.3342064531494
2808.3343255149425
2809.334444576735
2810.3345636385284
2811.3346827003215
2812.3348017621147
2813.334920823908
2814.335039885701
2815.3351589474937
2816.335278009287
2817.33539707108
2818.335516132873
2819.3356351946663
2820.335754256459
2821.335873318252
2822.3359923800454
2823.3361114418385
2824.3362305036317
2825.3363495654244
2826.3364686272175
2827.3365876890107
2828.336706750804
2829.336825812597
2830.3369448743897
2831.337063936183
2832.337182997976
2833.337302059769
2834.3374211215623
2835.3375401833555
2836.337659245148
2837.3377783069413
2838.3378973687345
2839.3380164305277
2840.338135492321
2841.3382545541135
2842.3383736159067
2843.3384926777
2844.338611739493
2845.338730801286
2846.338849863079
2847.338968924872
2848.339087986665
2849.3392070484583
2850.3393261102515
2851.339445172044
2852.3395642338373
2853.3396832956305
2854.3398023574237
2855.3

3611.429932134778
3612.430051196571
3613.4301702583643
3614.4302893201575
3615.43040838195
3616.4305274437434
3617.4306465055365
3618.4307655673297
3619.430884629123
3620.4310036909155
3621.4311227527087
3622.431241814502
3623.431360876295
3624.431479938088
3625.431598999881
3626.431718061674
3627.431837123467
3628.4319561852603
3629.4320752470535
3630.4321943088466
3631.4323133706393
3632.4324324324325
3633.4325514942257
3634.432670556019
3635.432789617812
3636.4329086796047
3637.433027741398
3638.433146803191
3639.433265864984
3640.4333849267773
3641.43350398857
3642.433623050363
3643.4337421121563
3644.4338611739495
3645.4339802357426
3646.4340992975353
3647.4342183593285
3648.4343374211217
3649.434456482915
3650.434575544708
3651.434694606501
3652.434813668294
3653.434932730087
3654.43505179188
3655.4351708536733
3656.4352899154665
3657.435408977259
3658.4355280390523
3659.4356471008455
3660.4357661626386
3661.435885224432
3662.4360042862245
3663.4361233480176
3664.436242409811
366

...

5002.595547088939
5003.5956661507325
5004.595785212526
5005.595904274319
5006.596023336112
5007.596142397904
5008.596261459697
5009.596380521491
5010.596499583284
5011.596618645077
5012.59673770687
5013.596856768663
5014.596975830456
5015.5970948922495
5016.597213954043
5017.597333015836
5018.597452077628
5019.597571139421
5020.597690201214
5021.597809263008
5022.597928324801
5023.598047386594
5024.598166448387
5025.59828551018
5026.598404571973
5027.5985236337665
5028.598642695559
5029.598761757352
5030.598880819145
5031.598999880938
5032.599118942731
5033.5992380045245
5034.599357066318
5035.599476128111
5036.599595189904
5037.599714251697
5038.59983331349
5039.599952375283
5040.600071437076
5041.600190498869
5042.600309560662
5043.600428622455
5044.600547684248
5045.6006667460415
5046.600785807835
5047.600904869628
5048.601023931421
5049.601142993213
5050.601262055006
5051.6013811167995
5052.601500178593
5053.601619240386
5054.601738302179
5055.601857363972
5056.601976425765
5057.60

5461.650196451958
5462.650315513752
5463.650434575545
5464.650553637338
5465.650672699131
5466.650791760924
5467.650910822717
5468.6510298845105
5469.651148946304
5470.651268008097
5471.651387069889
5472.651506131682
5473.651625193475
5474.651744255269
5475.651863317062
5476.651982378855
5477.652101440648
5478.652220502441
5479.652339564234
5480.6524586260275
5481.65257768782
5482.652696749613
5483.652815811406
5484.652934873199
5485.653053934992
5486.6531729967855
5487.653292058579
5488.653411120372
5489.653530182165
5490.653649243958
5491.653768305751
5492.653887367544
5493.654006429337
5494.65412549113
5495.654244552923
5496.654363614716
5497.654482676509
5498.6546017383025
5499.654720800096
5500.654839861889
5501.654958923682
5502.655077985474
5503.655197047267
5504.655316109061
5505.655435170854
5506.655554232647
5507.65567329444
5508.655792356233
5509.655911418026
5510.6560304798195
5511.656149541613
5512.656268603406
5513.656387665198
5514.656506726991
5515.656625788784
5516.656

5917.704488629599
5918.704607691392
5919.704726753185
5920.704845814978
5921.704964876772
5922.705083938565
5923.705203000357
5924.70532206215
5925.705441123943
5926.705560185736
5927.70567924753
5928.705798309323
5929.705917371116
5930.706036432909
5931.706155494702
5932.706274556495
5933.7063936182885
5934.706512680081
5935.706631741874
5936.706750803667
5937.70686986546
5938.706988927253
5939.707107989047
5940.70722705084
5941.707346112633
5942.707465174426
5943.707584236219
5944.707703298011
5945.707822359805
5946.707941421598
5947.708060483391
5948.708179545184
5949.708298606977
5950.70841766877
5951.7085367305635
5952.708655792357
5953.70877485415
5954.708893915943
5955.709012977735
5956.709132039528
5957.709251101322
5958.709370163115
5959.709489224908
5960.709608286701
5961.709727348494
5962.709846410287
5963.7099654720805
5964.710084533874
5965.710203595666
5966.710322657459
5967.710441719252
5968.710560781045
5969.7106798428385
5970.710798904632
5971.710917966425
5972.7110370

6376.759137992618
6377.759257054411
6378.759376116204
6379.759495177997
6380.759614239791
6381.759733301584
6382.759852363377
6383.75997142517
6384.760090486963
6385.760209548756
6386.76032861055
6387.760447672342
6388.760566734135
6389.760685795928
6390.760804857721
6391.760923919514
6392.761042981308
6393.761162043101
6394.761281104894
6395.761400166687
6396.76151922848
6397.7616382902725
6398.761757352066
6399.761876413859
6400.761995475652
6401.762114537445
6402.762233599238
6403.762352661031
6404.762471722825
6405.762590784618
6406.762709846411
6407.762828908204
6408.762947969996
6409.763067031789
6410.763186093583
6411.763305155376
6412.763424217169
6413.763543278962
6414.763662340755
6415.763781402548
6416.7639004643415
6417.764019526135
6418.764138587927
6419.76425764972
6420.764376711513
6421.764495773306
6422.7646148351
6423.764733896893
6424.764852958686
6425.764972020479
6426.765091082272
6427.765210144065
6428.7653292058585
6429.765448267651
6430.765567329444
6431.76568639

6832.8134301702585
6833.813549232052
6834.813668293845
6835.813787355638
6836.813906417431
6837.814025479224
6838.814144541017
6839.81426360281
6840.814382664603
6841.814501726396
6842.814620788189
6843.814739849982
6844.814858911775
6845.814977973569
6846.815097035362
6847.815216097155
6848.815335158948
6849.815454220741
6850.8155732825335
6851.815692344327
6852.81581140612
6853.815930467913
6854.816049529706
6855.816168591499
6856.816287653292
6857.816406715086
6858.816525776879
6859.816644838672
6860.816763900465
6861.816882962257
6862.81700202405
6863.817121085844
6864.817240147637
6865.81735920943
6866.817478271223
6867.817597333016
6868.817716394809
6869.817835456603
6870.817954518396
6871.818073580188
6872.818192641981
6873.818311703774
6874.818430765567
6875.818549827361
6876.818668889154
6877.818787950947
6878.81890701274
6879.819026074533
6880.819145136326
6881.8192641981195
6882.819383259912
6883.819502321705
6884.819621383498
6885.819740445291
6886.819859507084
6887.8199785

In [15]:
##run the sensitivity analysis with respect to the rates

def sens_rate(facet):    
    dk=1.0e-2
    heights_CO2,heights_CO=[],[]
    temps_CO2,temps_CO=[],[]
    
    #cycle through each reaction",surf.n_reactions
    for m in range(facet.n_reactions):
        # input file containing the surface reaction mechanism
        cti_file = 'original_mechanism_JPCC_2021/Multifacet_mechanism.yaml'
        # import the gas model and set the initial conditions
        gas = ct.Solution(cti_file, 'gas')
        gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

        #import the (111) surface
        surf111 = ct.Interface(cti_file,'surface1', [gas])
        surf111.TP = T, p*ct.one_atm
        surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

        #import the (211) surface
        surf211 = ct.Interface(cti_file,'surface2', [gas])
        surf211.TP = T, p*ct.one_atm
        surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

        #import the (100) surface
        surf100 = ct.Interface(cti_file,'surface3', [gas])
        surf100.TP = T, p*ct.one_atm
        surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

        #import the (110) surface
        surf110 = ct.Interface(cti_file,'surface4', [gas])
        surf110.TP = T, p*ct.one_atm
        surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

        #typical Cantera setup
        upstream = ct.Reservoir(gas, name='upstream')
        downstream = ct.Reservoir(gas, name='downstream')

        r=ct.IdealGasReactor(gas, energy='off')
        r.volume=volume
        rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
        rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
        rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
        rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

        #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
        vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
        mass_flow_rate =  vdot* gas.density
        mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
        v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

        sim=ct.ReactorNet([r])

        ##set relative and absolute tolerances on the simulation
        sim.rtol = 1.0e-12
        sim.atol = 1.0e-22
        sim.max_err_test_fails = 10000
        rxn_time=np.linspace(0,8400,8400)

        gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])
        
        print(facet.reaction_equations()[m])
        #reset all multipliers
        facet.set_multiplier(1.0)
        #perturb each reaction by dk
        facet.set_multiplier(1+dk, m)
            
        Ti=np.zeros(len(rxn_time))
        sim.max_time_step=1e-4
        #Run the simulation
        for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs[:,i]=gas.X
        
        ind=5410
    
        max_height_CO2=np.max(gas_mole_fracs[i_co2,ind:])*1e6
        print(max_height_CO2)
        max_Temp_CO2=ref_Temp_CO2-Ti[np.argmax(gas_mole_fracs[i_co2,ind:])+ind] 
    
        max_height_CO=np.max(gas_mole_fracs[i_co,ind:])*1e6
        max_Temp_CO=ref_Temp_CO-Ti[np.argmax(gas_mole_fracs[i_co,ind:])+ind]  
    
        heights_CO2.append(max_height_CO2)
        temps_CO2.append(max_Temp_CO2)

        heights_CO.append(max_height_CO)
        temps_CO.append(max_Temp_CO)
    
        #reset all multiplier
        facet.set_multiplier(1.0)
        
    return heights_CO2, heights_CO, temps_CO2, temps_CO

sens_rate(surf111)


CO2 + site(111) <=> CO2(111)
547.8440940244743
CO + site(111) <=> OC(111)
547.8440940244743
CO2(111) + site(111) <=> O(111) + OC(111)
547.8440940244743
C(111) + O(111) <=> OC(111) + site(111)


KeyboardInterrupt: 

In [21]:
##run the sensitivity analysis with respect to the rates

def sens_thermo(facet):
    heights_CO2,heights_CO=[],[]
    temps_CO2,temps_CO=[],[]
    
    dH=1.0e-0
    dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
    for count,j in enumerate(facet.species_names[1:]):
    
        # import the gas model and set the initial conditions
        gas = ct.Solution(cti_file, 'gas')
        gas.TPX = T, p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'

        #import the (111) surface
        surf111 = ct.Interface(cti_file,'surface1', [gas])
        surf111.TP = T, p*ct.one_atm
        surf111.coverages = {'site(111)':1 , 'O(111)':0.0, 'OC(111)':0.0 , 'CO2(111)':0}

        #import the (211) surface
        surf211 = ct.Interface(cti_file,'surface2', [gas])
        surf211.TP = T, p*ct.one_atm
        surf211.coverages = {'site(211)':1,  'O(211)':0.0, 'OC(211)':0.0 , 'CO2(211)':0}

        #import the (100) surface
        surf100 = ct.Interface(cti_file,'surface3', [gas])
        surf100.TP = T, p*ct.one_atm
        surf100.coverages = {'site(100)':1, 'O(100)':0.0, 'OC(100)':0.0, 'CO2(100)':0}

        #import the (110) surface
        surf110 = ct.Interface(cti_file,'surface4', [gas])
        surf110.TP = T, p*ct.one_atm
        surf110.coverages = {'site(110)':1, 'O(110)':0.0 , 'OC(110)':0.0  , 'CO2(110)':0}

        #typical Cantera setup
        upstream = ct.Reservoir(gas, name='upstream')
        downstream = ct.Reservoir(gas, name='downstream')

        r=ct.IdealGasReactor(gas, energy='off')
        r.volume=volume
        rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
        rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
        rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
        rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)

        #This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
        vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
        mass_flow_rate =  vdot* gas.density
        mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
        v = ct.PressureController(r, downstream, master=mflow, K=1e-9)

        sim=ct.ReactorNet([r])

        ##set relative and absolute tolerances on the simulation
        sim.rtol = 1.0e-12
        sim.atol = 1.0e-22
        sim.max_err_test_fails = 10000
        rxn_time=np.linspace(0,8400,8400)

        #get indices of the gas and surface species, (211) and (111) have the same order of species
        i_co2=gas.species_index('CO2')
        i_co=gas.species_index('CO')

        i_surf_Ni=surf111.species_index('site(111)')
        i_surf_CO=surf111.species_index('OC(111)')
        i_surf_O=surf111.species_index('O(111)')
        i_surf_C=surf111.species_index('C(111)')
        i_surf_CO2=surf111.species_index('CO2(111)')

        gas_mole_fracs_thermo=np.zeros([gas.n_species, len(rxn_time)])
    

        #get the index for each species
        idx=facet.species_index(j)
        #get the species 
        s=facet.species(idx)
        print(s)
        #call the original coefficients of the NASA polynomial
        original_coeffs = s.thermo.coeffs
        #create an empty array for the perturbation
        perturbed_coeffs = np.ones_like(original_coeffs)
        #fill the array initially with the original coeffs
        perturbed_coeffs[:] = original_coeffs
        #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
        perturbed_coeffs[6] = original_coeffs[6] - dk
        #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
        perturbed_coeffs[13] = original_coeffs[13] - dk
        #reset all multipliers
            
        #fit a new NASA polynomial through the data and update the thermo database
        s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
        facet.modify_species(idx, s) 
        
        Ti=np.zeros(len(rxn_time))
        sim.max_time_step=1e-4
        #Run the simulation
        for i in range(len(rxn_time)):
            time=rxn_time[i]
            #print(time)
            if time < 10:
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0.0, CO2:0.0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):   
                sim.max_time_step=1
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step=10
                upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                sim.max_time_step=10
                upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
                r.syncState()
             
            sim.reinitialize  
            sim.advance(time)
            Ti[i]=r.thermo.T
            gas_mole_fracs_thermo[:,i]=gas.X
        
        ind=5410
    
        max_height_CO2=np.max(gas_mole_fracs_thermo[i_co2,ind:])*1e6
        
        max_Temp_CO2=Ti[np.argmax(gas_mole_fracs_thermo[i_co2,ind:])+ind] 
    
        max_height_CO=np.max(gas_mole_fracs_thermo[i_co,ind:])*1e6
        max_Temp_CO=Ti[np.argmax(gas_mole_fracs_thermo[i_co,ind:])+ind]  
        print(max_height_CO2,max_Temp_CO2,max_height_CO,max_Temp_CO)
        heights_CO2.append(max_height_CO2)
        temps_CO2.append(max_Temp_CO2)

        heights_CO.append(max_height_CO)
        temps_CO.append(max_Temp_CO)
    
        #reset all multiplier
        #restore the original NASA polynomial and perturb the next species (only one at a time)
        s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
        facet.modify_species(idx, s) 
        
    return heights_CO2, heights_CO, temps_CO2, temps_CO

sens_thermo(surf111)

<Species O(211)>
547.8440940244743 420.2262570940987 182.3048386021555 651.9205064094933
<Species OC(211)>
547.8440940244743 420.2262570940987 182.3048386021555 651.9205064094933
<Species C(211)>


KeyboardInterrupt: 